1) Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
Также должны поддерживаться следующие операции:
•	извлечение элемента по индексу;
•	устанавка нового значения по индексу;
•	получение текущей длины массива.


In [1]:
class DynamicArray:
    def __init__(self):
        self.array = []
    
    def append(self, element):
        self.array.append(element)
    
    def get(self, index):
        return self.array[index]
    
    def set(self, index, value):
        self.array[index] = value
    
    def remove_at(self, index):
        del self.array[index]
    
    def length(self):
        return len(self.array)
    
    def __str__(self):
        return str(self.array)

# Пример использования
dynamic_array = DynamicArray()
dynamic_array.append(1)
dynamic_array.append(2)
dynamic_array.append(3)

print(dynamic_array)
print(dynamic_array.get(1))
dynamic_array.set(1, 5)
print(dynamic_array)
dynamic_array.remove_at(0)
print(dynamic_array)
print(dynamic_array.length())


[1, 2, 3]
2
[1, 5, 3]
[5, 3]
2


1.1) Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print.

In [2]:
from collections import deque

def queue_generator(queue1, queue2):
    while queue1 or queue2:
        if queue1:
            yield queue1.pop()
        if queue2:
            yield queue2.pop()

# Пример использования
queue1 = deque([1, 2, 3])
queue2 = deque([4, 5, 6, 7])

gen = queue_generator(queue1, queue2)
for val in gen:
    print(val)

3
7
2
6
1
5
4


2) Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.

In [5]:
from collections import deque

def deque_generator(queue1, queue2):
    while queue1 or queue2:
        if queue1:
            yield queue1.pop() if queue1 else None
        if queue2:
            yield queue2.pop() if queue2 else None

# Пример использования
queue1 = deque([1, 2, 3])
queue2 = deque([4, 5, 6, 7])

gen = deque_generator(queue1, queue2)
for val in gen:
    if val is not None:
        print(val)


3
7
2
6
1
5
4


3) Реализовать классы с медодом action():
а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).


In [8]:
from queue import Queue

class Pump:
    def __init__(self, queue, generator):
        self.queue = queue
        self.generator = generator

    def action(self):
        try:
            value = next(self.generator)
            self.queue.put(value)
            print(f"Added value {value} to the queue")
        except StopIteration:
            print("Generator is exhausted")

# Пример использования класса Pump
def simple_generator():
    for i in range(5):
        yield i

queue = Queue()
pump = Pump(queue, simple_generator())

for _ in range(6):  # Пытаемся добавить больше значений, чем есть в генераторе
    pump.action()

Added value 0 to the queue
Added value 1 to the queue
Added value 2 to the queue
Added value 3 to the queue
Added value 4 to the queue
Generator is exhausted


b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.

In [9]:
class MultiAction:
    def __init__(self, class_instance, n):
        self.class_instance = class_instance
        self.n = n

    def action(self):
        for _ in range(self.n):
            self.class_instance.action()

# Пример использования класса MultiAction
class ExampleClass:
    def __init__(self, name):
        self.name = name
        self.counter = 0

    def action(self):
        self.counter += 1
        print(f"{self.name} - Action performed ({self.counter} times)")

example_instance = ExampleClass("Example")
multi_action = MultiAction(example_instance, 3)

multi_action.action()

Example - Action performed (1 times)
Example - Action performed (2 times)
Example - Action performed (3 times)


c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.

In [10]:
from queue import Queue

class MultiPump:
    def __init__(self, queues, generator):
        self.queues = queues
        self.generator = generator
        self.current_queue_index = 0

    def action(self):
        try:
            value = next(self.generator)
            current_queue = self.queues[self.current_queue_index]
            current_queue.put(value)
            self.current_queue_index = (self.current_queue_index + 1) % len(self.queues)
        except StopIteration:
            print("Generator has no more values.")

# Пример использования класса MultiPump
queues = [Queue(), Queue(), Queue()]
gen = (i for i in range(10))  # Генератор значений от 0 до 9

multi_pump = MultiPump(queues, gen)

# Вызываем метод action() несколько раз для заполнения очередей
for _ in range(10):
    multi_pump.action()

# Выводим содержимое каждой из очередей
for i, queue in enumerate(queues):
    print(f"Queue {i}:")
    while not queue.empty():
        print(queue.get())

Queue 0:
0
3
6
9
Queue 1:
1
4
7
Queue 2:
2
5
8


4) При помощи GenFromQ, Pump реализовать систему опработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).



In [ ]:
import queue
import random
import threading
import time

class GenFromQ:
    def __init__(self, queue):
        self.queue = queue

    def generate_message(self):
        raise NotImplementedError("Subclasses must implement generate_message method.")

    def run(self):
        while True:
            message = self.generate_message()
            self.queue.put(message)
            time.sleep(1)  # Имитация создания сообщения

class Pump(threading.Thread):
    def __init__(self, generator, name):
        super().__init__()
        self.generator = generator
        self.name = name

    def run(self):
        print(f"Pump {self.name} started.")
        self.generator.run()
        print(f"Pump {self.name} stopped.")
class MessageGenerator(GenFromQ):
    def generate_message(self):
        messages = ["Сообщение 1", "Сообщение 2", "Сообщение 3"]
        return random.choice(messages)

queue1 = queue.Queue()
generator1 = MessageGenerator(queue1)

queue2 = queue.Queue()
generator2 = MessageGenerator(queue2)

queue3 = queue.Queue()
generator3 = MessageGenerator(queue3)
class MultiPump:
    def __init__(self, *queues):
        self.queues = queues

    def run(self):
        while True:
            for q in self.queues:
                if not q.empty():
                    message = q.get()
                    print(f"Received message: {message}")

# Создаем экземпляр MultiPump и добавляем все три очереди
multi_pump = MultiPump(queue1, queue2, queue3)
pump1 = Pump(generator1, "Pump 1")
pump2 = Pump(generator2, "Pump 2")
pump3 = Pump(generator3, "Pump 3")

# Запускаем насосы
pump1.start()
pump2.start()
pump3.start()

# Запускаем MultiPump
multi_pump.run()



Pump Pump 1 started.
Pump Pump 2 started.
Pump Pump 3 started.
Received message: Сообщение 2
Received message: Сообщение 2
Received message: Сообщение 2
Received message: Сообщение 3
Received message: Сообщение 2
Received message: Сообщение 2
Received message: Сообщение 3
Received message: Сообщение 2
Received message: Сообщение 3
Received message: Сообщение 3
Received message: Сообщение 1
Received message: Сообщение 3
Received message: Сообщение 2
Received message: Сообщение 3
Received message: Сообщение 1
Received message: Сообщение 1
Received message: Сообщение 2
Received message: Сообщение 1
Received message: Сообщение 2
Received message: Сообщение 1
Received message: Сообщение 2
Received message: Сообщение 2
Received message: Сообщение 2
Received message: Сообщение 3
Received message: Сообщение 1
Received message: Сообщение 3
Received message: Сообщение 3
Received message: Сообщение 3
Received message: Сообщение 1
Received message: Сообщение 3
Received message: Сообщение 1
Receive